In [1]:
from transformers import AutoModel,AutoTokenizer , Trainer,TrainingArguments,BitsAndBytesConfig
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
import deepspeed
import os
import pandas as pd

cache_dir='/proj/ciptmp/ix05ogym/.cache/'


/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No ROCm runtime is found, using ROCM_HOME='/usr'


[2024-06-11 20:27:52,972] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.1), only 1.0.0 is known to be compatible


In [2]:
mydata = pd.read_pickle('cleaned.pkl')
mydata.reset_index(inplace=True)
mydata:pd.DataFrame
def myf(x):
    print(x)
    #input()

print(mydata['text'][1])



*<input data-testid="textinput" id="email" required="" type="email" autocomplete="email" name="email" class="sc-bsgiji jskkmm" value="">*<input data-testid="firstname" id="firstname" required="" name="firstname" type="text" class="sc-kilemz pddck" value="">*<input data-testid="lastname" id="lastname" required="" name="lastname" type="text" class="sc-kilemz pddck" value="">documents:.doc, .docx, .pdf, .rtf, .txt<input accept=".doc, .docx, .pdf, .rtf, .txt" type="file" autocomplete="off" tabindex="-1" class="fileuploader___styledinput-sc-e2002db3-0 djcsgj">cv*<input accept=".doc, .docx, .pdf, .rtf, .txt" type="file" autocomplete="off" tabindex="-1" class="fileuploader___styledinput-sc-e2002db3-0 djcsgj">cover letteroptionalterms & conditionsprivacy policyapply now<button type="submit" class="sc-jotjna fnrczm">apply now</button>


In [3]:
tokenizer = AutoTokenizer.from_pretrained("McGill-NLP/Llama-3-8B-Web")
x = tokenizer("[[xpath]]")
tokenizer.convert_ids_to_tokens(x['input_ids'])
tokenizer.additional_special_tokens

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[]

In [4]:
weblinx_dataset = datasets.load_dataset('McGill-NLP/WebLINX',cache_dir=cache_dir)
weblinx_dataset.keys()


dict_keys(['train', 'validation', 'test', 'test_iid', 'test_geo', 'test_vis', 'test_cat', 'test_web'])

In [5]:
train_dataset = weblinx_dataset['train']
def myfind(x):
    if isinstance(x,str):
        if x.find('resume')!=-1:
            #print(x)
            return True
    return False
    
job = train_dataset.filter(myfind,input_columns='utterances')
print(job)
job['candidates'][0]

Dataset({
    features: ['demo', 'turn', 'action', 'action_history', 'utterances', 'candidates', 'clean_html', 'viewport'],
    num_rows: 75
})


"(uid = 0b6ec548-30c4-4dfa) [[tag]] div [[xpath]] /html/body/div[1]/...div[2]/div/div[1] [[bbox]] x=230 y=210 width=767 height=370 [[attributes]] data-event-id='19' data... font-size: 16px;' [[children]] div div \n(uid = fc30e488-ea91-4030) [[tag]] div [[xpath]] /html/body/div[1]/...div/div[1]/div[1] [[text]] Dear\xa0 [[bbox]] x=230 y=378 width=751 height=18 [[attributes]] dir='ltr' data-setdir='...ea91-4030' [[children]] span \n(uid = 27ae19c7-8f93-499e) [[tag]] span [[xpath]] /html/body/div[1]/div...div[1]/div[1]/span [[text]] Team, [[bbox]] x=230 y=360 width=95.2 height=17 [[attributes]] data-webtasks-id='27...8f93-499e' \n(uid = 40d97e2a-ad43-48d2) [[tag]] div [[xpath]] /html/body/div[1]/div...div[1]/div/div/div [[bbox]] x=194 y=111 width=839 height=538 [[attributes]] role='main' data-test-id='... P_ZzJed p_R ' [[children]] ul div \n(uid = 89c6ffef-8b3d-47df) [[tag]] a [[xpath]] /html/body/div[1]/div...nav/div/div[1]/a [[text]] Compose [[bbox]] x=16 y=127 width=160 height=36 [[attr

In [6]:
model_name = "PrunaAI/McGill-NLP-Llama-3-8B-Web-HQQ-4bit-smashed"
#config = BitsAndBytesConfig(
    #load_in_4bit=True,

    #load_in_8bit=True,
    
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    #)

"""
model = AutoModel.from_pretrained(model_name,
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir)


print(model)
"""
#https://huggingface.co/docs/transformers/main/en/quantization/hqq
try:
 model_hqq = HQQModelForCausalLM.from_quantized(model_name, device_map='auto',cache_dir=cache_dir).half()#.to(torch.bfloat16) #.half()
except: 
 model_hqq = AutoHQQHFModel.from_quantized(model_name,cache_dir=cache_dir)
 
 



100%|██████████| 225/225 [00:00<00:00, 19649.34it/s]


In [7]:
#model = model.eval()
#input_ids = tokenizer("create an html apply button?", return_tensors='pt').to('cuda')["input_ids"]
#outputs = model.generate(input_ids, max_new_tokens=216)
#tokenizer.decode(outputs[0])

In [8]:
print(model_hqq)


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): HQQLinear(in_features=4096, out_features=4096, bias=False)
          (k_proj): HQQLinear(in_features=4096, out_features=1024, bias=False)
          (v_proj): HQQLinear(in_features=4096, out_features=1024, bias=False)
          (o_proj): HQQLinear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): HQQLinear(in_features=4096, out_features=14336, bias=False)
          (up_proj): HQQLinear(in_features=4096, out_features=14336, bias=False)
          (down_proj): HQQLinear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


In [9]:
"""
{
  "train_batch_size": 8,
  "gradient_accumulation_steps": 1,
  "optimizer": {
    "type": "Adam",
    "params": {
      "lr": 0.00015
    }
  },
  "fp16": {
    "enabled": true
  },
  "zero_optimization": true
}
"""
#os.environ['DS_SKIP_CUDA_CHECK'] = '1'

"""
  "optimizer": {
      "type": "AdamW",
      "params": {
        "lr": 2e-5,
        "weight_decay": 0.0,
        "bias_correction": True
      }
    },

"""
ds_config = {
    "train_batch_size": 1,
    "gradient_accumulation_steps": 1,
    "gradient_clipping": 1.0,
       
  
   """ "quantize_training": {
      "enabled": True,
      "quantize_verbose": True,
      "quantizer_kernel": True,
      "quantize-algo": {
        "q_type": "symmetric"
      },
      "quantize_bits": {
        "start_bits": 4,
        "target_bits": 2
      },
      "quantize_schedule": {
        "quantize_period": 400,
        "schedule_offset": 0
      },
      #"quantize_groups": 8,
    },"""
  
    
    "bf16": {
        "enabled": False,
        #"initial_scale_power": 16,

    },
    "zero_optimization": {
        "stage": 3,
      "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
          "offload_param": {
            "device": "cpu",
            "pin_memory": True
        },
        
    
    },
    "zero_force_ds_cpu_optimizer": False,
   
   
}
"""
"activation_checkpointing": {
    "partition_activations": True,
    "cpu_checkpointing": True
},
    "offload_optimizer": {
            "device": "cpu",
            "pin_memory": True
        },
          "offload_param": {
            "device": "cpu",
            "pin_memory": True
        },
"""
#os.environ["PATH"] = '/usr/bin/g++'
#os.environ["DEFAULT_TORCH_EXTENSION_PATH "] = '/proj/ciptmp/ix05ogym/tmp'
#os.environ["TMPDIR"]="/proj/ciptmp/ix05ogym/tmp"
# Move the optimizer instance to the GPU
#optimizer = optimizer.to("cuda")

#op = torch.optim.Adam
"""
model, optimizer, _, _ = deepspeed.initialize(
    model=model_hqq,
    model_parameters=model_hqq.parameters(),
    #optimizer=torch.optim.Adam, #deepspeed.ops.adam.DeepSpeedCPUAdam
    config=ds_config
    
)
"""
model = model_hqq

In [10]:
snapshot_download("McGill-NLP/WebLINX", repo_type="dataset", allow_patterns="templates/*",cache_dir=cache_dir)
valid = datasets.load_dataset("McGill-NLP/WebLINX", split="validation[:5]",cache_dir=cache_dir)
template = """<s>[INST] <<SYS>>
{clean_html}
You will find above the HTML elements that are available for the current webpage.
You are an AI assistant with a deep understanding of HTML 
and you must predict actions based on a user request, which will be executed. 
Use one of the following, replacing [] with an appropriate value: change(value=[str], uid=[str]) ; click(uid=[str]) ; load(url=[str]) ; say(speaker="navigator", utterance=[str]) ; scroll(x=[int], y=[int]) ; 
submit(uid=[str]) ; text_input(text=[str], uid=[str]) ;
The user's first and last 4 utterances are:
{utterances} ;
Viewport size: {viewport}
Only the last 5 turns are provided.
Here are the top candidates for this turn:
{candidates}

<</SYS>>[/INST]
{action_history}
Please select the best action using the correct format, do not provide any other information or explanation.
[/INST]"""


Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00, 1859.18it/s]


In [11]:
template = """<s>[INST] <<SYS>>
{clean_html}
You will find above the HTML elements that are available for the current webpage.
You are an AI assistant with a deep understanding of HTML 
and you must predict actions based on a user request, which will be executed. 
Use one of the following, replacing [] with an appropriate value: change(value=[str], uid=[str]) ; click(uid=[str]) ; load(url=[str]) ; say(speaker="navigator", utterance=[str]) ; scroll(x=[int], y=[int]) ; 
submit(uid=[str]) ; text_input(text=[str], uid=[str]) ;
The user's first and last 4 utterances are:
{utterances} ;
Viewport size: {viewport}
Only the last 5 turns are provided.
Here are the top candidates for this turn:
{candidates}

<</SYS>>[/INST]
{action_history}
Please select the best action using the correct format, do not provide any other information or explanation.
[/INST]"""


In [33]:


i=1

l = mydata[mydata['text']==mydata['text'][i]]['element'].tolist()
s=''
for j in range(len(l)):
    p = l[j].split()
    pp = ''.join(p[1:-1])
    s += f"(uid: {j}) [[tag]] {p[0][1:]} [[attribute]] {pp} "
    
s

'(uid: 0) [[tag]] input [[attribute]] data-testid="textinput"id="email"required=""type="email"autocomplete="email"name="email"class="sc-bsgijijskkmm" (uid: 1) [[tag]] input [[attribute]] data-testid="firstname"id="firstname"required=""name="firstname"type="text"class="sc-kilemzpddck" (uid: 2) [[tag]] input [[attribute]] data-testid="lastname"id="lastname"required=""name="lastname"type="text"class="sc-kilemzpddck" (uid: 3) [[tag]] input [[attribute]] accept=".doc,.docx,.pdf,.rtf,.txt"type="file"autocomplete="off"tabindex="-1"class="fileuploader___styledinput-sc-e2002db3-0 (uid: 4) [[tag]] button [[attribute]] type="submit"class="sc-jotjnafnrczm">apply '

In [13]:
d = {"clean_html":mydata['text'][i],
 "utterances":"here is my email: majid@gmail.com and my password: majidpassword please fill the correct input and submit",
 "candidates":s,
 "action_history":""
 }
mytemplate = """<s>[INST] <<SYS>>
{clean_html}
You will find above the HTML elements that are available for the current webpage.
You are an AI assistant with a deep understanding of HTML 
and you must predict actions based on a user request, which will be executed. 
Use one of the following, replacing [] with an appropriate value: change(value=[str], uid=[str]) ; click(uid=[str]) ; load(url=[str]) ; say(speaker="navigator", utterance=[str]) ; scroll(x=[int], y=[int]) ; 
submit(uid=[str]) ; text_input(text=[str], uid=[str]) ;
The user's first and last 4 utterances are:
{utterances} ;
Only the last 5 turns are provided.
Here are the top candidates for this turn:
{candidates}

<</SYS>>[/INST]
{action_history}
Please select the best action using the correct format, do not provide any other information or explanation.
[/INST]"""

state = mytemplate.format(**d)
print(state)


<s>[INST] <<SYS>>
*<input data-testid="textinput" id="email" required="" type="email" autocomplete="email" name="email" class="sc-bsgiji jskkmm" value="">*<input data-testid="firstname" id="firstname" required="" name="firstname" type="text" class="sc-kilemz pddck" value="">*<input data-testid="lastname" id="lastname" required="" name="lastname" type="text" class="sc-kilemz pddck" value="">documents:.doc, .docx, .pdf, .rtf, .txt<input accept=".doc, .docx, .pdf, .rtf, .txt" type="file" autocomplete="off" tabindex="-1" class="fileuploader___styledinput-sc-e2002db3-0 djcsgj">cv*<input accept=".doc, .docx, .pdf, .rtf, .txt" type="file" autocomplete="off" tabindex="-1" class="fileuploader___styledinput-sc-e2002db3-0 djcsgj">cover letteroptionalterms & conditionsprivacy policyapply now<button type="submit" class="sc-jotjna fnrczm">apply now</button>
You will find above the HTML elements that are available for the current webpage.
You are an AI assistant with a deep understanding of HTML 
and

In [47]:
realstate = template.format(**valid[0])
#print(state)

In [15]:
#state = template.format(**valid[0])
#print(state)

In [44]:
yy =f"what is the third uid? {s}"
yy

'what is the third uid? (uid: 0) [[tag]] input [[attribute]] data-testid="textinput"id="email"required=""type="email"autocomplete="email"name="email"class="sc-bsgijijskkmm" (uid: 1) [[tag]] input [[attribute]] data-testid="firstname"id="firstname"required=""name="firstname"type="text"class="sc-kilemzpddck" (uid: 2) [[tag]] input [[attribute]] data-testid="lastname"id="lastname"required=""name="lastname"type="text"class="sc-kilemzpddck" (uid: 3) [[tag]] input [[attribute]] accept=".doc,.docx,.pdf,.rtf,.txt"type="file"autocomplete="off"tabindex="-1"class="fileuploader___styledinput-sc-e2002db3-0 (uid: 4) [[tag]] button [[attribute]] type="submit"class="sc-jotjnafnrczm">apply '

In [45]:
tokenized_state = tokenizer(yy,return_tensors='pt',truncation=True,max_length=512).to('cuda')#.to(torch.bfloat16)
print(tokenized_state)
print(tokenized_state['input_ids'].shape)
#model = model.eval()
#with torch.no_grad():
#    o=model(**tokenized_state)
    #del tokenized_state
    #torch.cuda.empty_cache()
    
#o
#print("Action:", out['generated_text'])

{'input_ids': tensor([[12840,   374,   279,  4948, 14959,    30,   320,  2480,    25,   220,
            15,     8,  4416,  4681,  5163,  1988,  4416,  9294,  5163,   828,
         90352,   429,  1342,  1379, 29800,   429,  2386,     1,  6413,  8573,
          1337,   429,  2386,     1, 57614,   429,  2386, 32586,   429,  2386,
         31508,   429,  2445, 57530,    70, 35973,  2580, 19747,  3906,     1,
           320,  2480,    25,   220,    16,     8,  4416,  4681,  5163,  1988,
          4416,  9294,  5163,   828, 90352,   429, 29525, 29800,   429, 29525,
             1,  6413,  8573,   609,   429, 29525, 45570,   429,  1342, 31508,
           429,  2445, 12934,   458,    76, 94869,   634,   377,     1,   320,
          2480,    25,   220,    17,     8,  4416,  4681,  5163,  1988,  4416,
          9294,  5163,   828, 90352,   429, 30306, 29800,   429, 30306,     1,
          6413,  8573,   609,   429, 30306, 45570,   429,  1342, 31508,   429,
          2445, 12934,   458,    76, 9

In [17]:
y =o.logits.cpu()

In [18]:
m = y.argmax(2)
print(m.shape)
pp=tokenizer.decode(m[0])



torch.Size([1, 8])


In [19]:
print(pp)

<|start_header_id|> the difference of theawns? The


In [46]:
outputs = model.generate(tokenized_state['input_ids'], max_new_tokens=128)
o = tokenizer.decode(outputs[0])
print(o)

what is the third uid? (uid: 0) [[tag]] input [[attribute]] data-testid="textinput"id="email"required=""type="email"autocomplete="email"name="email"class="sc-bsgijijskkmm" (uid: 1) [[tag]] input [[attribute]] data-testid="firstname"id="firstname"required=""name="firstname"type="text"class="sc-kilemzpddck" (uid: 2) [[tag]] input [[attribute]] data-testid="lastname"id="lastname"required=""name="lastname"type="text"class="sc-kilemzpddck" (uid: 3) [[tag]] input [[attribute]] accept=".doc,.docx,.pdf,.rtf,.txt"type="file"autocomplete="off"tabindex="-1"class="fileuploader___styledinput-sc-e2002db3-0 (uid: 4) [[tag]] button [[attribute]] type="submit"class="sc-jotjnafnrczm">apply  [[children]] span 
(uid: 5) [[tag]] div [[attribute]] data-webtasks-id="5" class="sc-1kfofo2-0 eQYjR" [[children]] div 
(uid: 6) [[tag]] div [[attribute]] data-webtasks-id="6" class="sc-1kfofo2-0 eQYjR" [[children]] div 
(uid: 7) [[tag]] div [[attribute]] data-webtasks-id="7" class="sc-1kfofo2-0 eQYjR" [[children]] d

In [54]:
from transformers import pipeline
agent = pipeline("",model=model,tokenizer=tokenizer,device='cuda', torch_dtype='auto')
out = agent(yy, return_full_text=False)

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Input length of input_ids is 210, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


In [55]:
out

[{'generated_text': ' [['}]